In [1]:
import numpy as np

In [2]:
## 랜덤 변수 설정 (n_features : 특성의 개수 , n_values : 데이터 개수 )

random_generator = np.random.default_rng()

def generate_data(n_features, n_values):
    features = random_generator.random((n_features, n_values))
    targets = random_generator.random((n_features))
    return features, targets

In [3]:
x_train, y_train = generate_data(500,4)
x_test, y_test = generate_data(5,4)

In [9]:
x_train.shape

(500, 4)

In [6]:
class RNN:
    def __init__(self):
        self.global_weight = [1, 1] # [Input, Recurrent Weight]
        self.local_weight = [0.001, 0.001] ## RNN 입력이 두개이기 때문 (현재, 과거 데이터)
        self.W_sign = [0, 0] # penalty 용도

        self.eta_p = 1.2
        self.eta_n = 0.5

    # 입력값 설정
    def state_handler(self, input_x, previous_state):
        return input_x * self.global_weight[0] + previous_state * self.global_weight[1]

    # 순방향 전파
    def forward_propagation(self, X):

      S = np.zeros((X.shape[0], X.shape[1]+1)) ## 초기화
      for k in range(0, X.shape[1]):  ## 실제 입력
          next_state = self.state_handler(X[:,k], S[:,k]) ## 마지막 State와 입력을 합쳐 전달
          S[:,k+1] = next_state
      return S
    # 역전파
    def backward_propagation(self, X, S, grad_out):
   
        grad_over_time = np.zeros(( X.shape[0], X.shape[1]+1 ))
        grad_over_time[:,-1] = grad_out

        ## 2차원 데이터에 의한 x, y 가중치
        wx_grad = 0
        wy_grad = 0
        for k in range(X.shape[1], 0, -1):
            wx_grad += np.sum( grad_over_time[:, k] * X[:, k-1] )
            wy_grad += np.sum( grad_over_time[:, k] * S[:, k-1] )

            grad_over_time[:, k-1] = grad_over_time[:, k] * self.global_weight[1]
        return (wx_grad, wy_grad), grad_over_time

    def update_rprop(self, X, Y, W_prev_sign, local_weight):
        S = self.forward_propagation(X)
        grad_out = 2 * (S[:, -1] - Y) / 500
        W_grads, _ = self.backward_propagation(X, S, grad_out)
        self.W_sign = np.sign(W_grads)

        for i, _ in enumerate(self.global_weight):
            if self.W_sign[i] == W_prev_sign[i]:
                local_weight[i] *= self.eta_p
            else:
                local_weight[i] *= self.eta_n
        self.local_weight = local_weight

    def train(self, X, Y, training_epochs):
        for epochs in range(training_epochs):
            self.update_rprop(X, Y, self.W_sign, self.local_weight)

            for i, _ in enumerate(self.global_weight):
                self.global_weight[i] -= self.W_sign[i] * self.local_weight[i]

In [8]:
rnn = RNN()
rnn.train(x_train, y_train, 200)

print (f"Targets are: {y_test}")
y = rnn.forward_propagation(x_test)[:, -1]
print (f"Predicted are: {y}")

Targets are: [0.10439912 0.40632159 0.70744394 0.94351491 0.51583449]
Predicted are: [0.3790664  0.50918187 0.60429245 0.52113766 0.61947291]
